# Compute conformer energies for a small molecule

This notebook illustrates reading conformers of a molecule from an SDF file and computation of vacuum conformer energies using a SMIRNOFF force field.

Note that absolute vacuum potential energies can be sensitive to small changes in partial charge, for example due to using OpenEye or AmberTools to generate AM1-BCC charges. However, in our experience, _relative_ conformer energies are fairly consistent between AM1-BCC implementations.

Note also that the Open Force Field Toolkit produces deterministic charges that do not depend on the input conformation of parameterized molecules. See the [FAQ](https://open-forcefield-toolkit.readthedocs.io/en/stable/faq.html#the-partial-charges-generated-by-the-toolkit-don-t-seem-to-depend-on-the-molecule-s-conformation-is-this-a-bug) for more information.

In [1]:
import openmm
from openff.interchange import Interchange
from openff.units.openmm import from_openmm, to_openmm
from openmm import unit as openmm_unit
from rdkit.Chem import rdMolAlign

from openff.toolkit import ForceField, Molecule
from openff.toolkit.utils import get_data_file_path

In [2]:
loaded_molecules = Molecule.from_file(
    get_data_file_path("molecules/ruxolitinib_conformers.sdf"),
)

# Normalize to list
try:
    loaded_molecules = [*loaded_molecules]
except TypeError:
    loaded_molecules = [loaded_molecules]

# from_file loads each entry in the SDF into its own molecule,
# so collapse conformers into molecules
molecules = []
for next_molecule in loaded_molecules:
    if next_molecule in molecules:
        saved_molecule = molecules[molecules.index(next_molecule)]
        for conformer in next_molecule.conformers:
            saved_molecule.add_conformer(conformer)
    else:
        molecules.append(next_molecule)

print(
    f"Loaded {molecules[0].n_conformers} conformers"
    + f" of {molecules[0].to_smiles(explicit_hydrogens=False)!r}"
    + f"\n       and {len(molecules) - 1} other molecules"
)
molecules[0]

Loaded 10 conformers of 'c1nc(c2c(n1)NC=C2)C3=CN(N=C3)[C@H](CC#N)C4CCCC4'
       and 0 other molecules


NGLWidget(max_frame=9)

In [3]:
# Load the openff-2.1.0 force field appropriate for vacuum calculations (without constraints)
forcefield = ForceField("openff_unconstrained-2.1.0.offxml")

In [4]:
for molecule in molecules:
    if not molecule.name:
        molecule.name = molecule.to_hill_formula()
    n_confs = molecule.n_conformers
    print(f"{molecule.name}: {n_confs} conformers")

    print(f"Parametrizing {molecule.name} (may take a moment to calculate charges)")
    interchange = Interchange.from_smirnoff(forcefield, [molecule])

    integrator = openmm.VerletIntegrator(1 * openmm_unit.femtoseconds)
    simulation = interchange.to_openmm_simulation(integrator)

    # Make a temporary copy of the molecule that we can update for each minimization
    mol_copy = Molecule(molecule)

    # Print text header
    print("Conformer         Initial PE        Minimized PE        RMSD")
    output = [
        [
            "Conformer",
            "Initial PE (kcal/mol)",
            "Minimized PE (kcal/mol)",
            "RMSD between initial and minimized conformer (Angstrom)",
        ]
    ]
    for conformer_index, conformer in enumerate(molecule.conformers):
        # Set positions and get initial energy
        simulation.context.setPositions(to_openmm(conformer))
        orig_potential = simulation.context.getState(
            getEnergy=True
        ).getPotentialEnergy()

        # Perform the minimization
        simulation.minimizeEnergy()

        # Energy minimisation is done, but let's thoroughly record what we got

        min_state = simulation.context.getState(getEnergy=True, getPositions=True)
        min_potential = min_state.getPotentialEnergy()
        min_coords = from_openmm(min_state.getPositions())

        mol_copy.add_conformer(conformer)

        # Save the minimized conformer to file
        del mol_copy.conformers[:]
        mol_copy.add_conformer(min_coords)
        mol_copy.to_file(
            f"{molecule.name}_conf{conformer_index+1}_minimized.sdf",
            file_format="sdf",
        )

        # Calculate the RMSD between the initial and minimized conformer
        mol_copy.add_conformer(conformer)
        rdmol = mol_copy.to_rdkit()
        rmslist = []
        rdMolAlign.AlignMolConformers(rdmol, RMSlist=rmslist)
        minimization_rms = rmslist[0]

        # Record the results
        output.append(
            [
                conformer_index + 1,
                orig_potential.value_in_unit(openmm_unit.kilocalories_per_mole),
                min_potential.value_in_unit(openmm_unit.kilocalories_per_mole),
                minimization_rms,
            ]
        )
        print(
            f"{{:5d}} / {n_confs:5d} :  {{:8.3f}} kcal/mol {{:8.3f}} kcal/mol {{:8.3f}} Å".format(
                *output[-1]
            )
        )

    # Write the results out to CSV
    with open(f"{molecule.name}.csv", "w") as of:
        of.write(", ".join(output.pop(0)) + "\n")
        for line in output:
            of.write("{}, {:.3f}, {:.3f}, {:.3f}".format(*line) + "\n")

ruxolitinib: 10 conformers
Parametrizing ruxolitinib (may take a moment to calculate charges)


Conformer         Initial PE        Minimized PE        RMSD
    1 /    10 :   -62.294 kcal/mol  -95.481 kcal/mol    0.505 Å
    2 /    10 :   -55.582 kcal/mol  -92.723 kcal/mol    0.490 Å
    3 /    10 :   -17.811 kcal/mol  -92.900 kcal/mol    0.891 Å
    4 /    10 :   -53.680 kcal/mol  -94.485 kcal/mol    0.756 Å
    5 /    10 :   -54.243 kcal/mol  -92.087 kcal/mol    0.842 Å
    6 /    10 :   -56.083 kcal/mol  -94.656 kcal/mol    0.594 Å
    7 /    10 :   -53.253 kcal/mol  -92.893 kcal/mol    1.031 Å
    8 /    10 :   -63.596 kcal/mol  -94.760 kcal/mol    0.364 Å
    9 /    10 :   -60.427 kcal/mol  -95.257 kcal/mol    0.783 Å
   10 /    10 :   -55.957 kcal/mol  -95.356 kcal/mol    0.682 Å
